In [1]:
# Imports
import pandas as pd
import urllib3
import certifi
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
import datetime
import time
import numpy as np

### Scrapping

In [2]:
# Make PoolManager instance
http = urllib3.PoolManager(
        cert_reqs='CERT_REQUIRED',
        ca_certs=certifi.where())

# Make request
link = 'http://live.sts-timing.pl/mp2018/wyniki.php?search=1&dystans=1&dystans=1&filter%5Bcountry%5D=&filter%5Bcity%5D=&filter%5Bteam%5D=&filter%5Bsex%5D=&filter%5Bcat%5D=&show%5B%5D=1&show%5B%5D=2&show%5B%5D=3&show%5B%5D=4&show%5B%5D=5&show%5B%5D=6&show%5B%5D=7&show%5B%5D=8&show%5B%5D=9&show%5B%5D=10&show%5B%5D=11&show%5B%5D=12&show%5B%5D=13&show%5B%5D=14&show%5B%5D=15&show%5B%5D=16&show%5B%5D=17&show%5B%5D=18&show%5B%5D=19&sort='
r = http.request('GET', link)

In [3]:
# Parse data
soup = BeautifulSoup(r.data, 'html.parser')
table = soup.find('table')
table.prettify()[:750]

'<table class="table table-bordered table-condensed table-striped table-responsive" style="font-size:10px;">\n <thead>\n  <tr>\n   <th>\n    #\n   </th>\n   <th>\n    Numer\n   </th>\n   <th>\n    Imię i nazwisko\n   </th>\n   <th>\n    Miasto\n   </th>\n   <th>\n    Kraj\n   </th>\n   <th>\n    Team\n   </th>\n   <th>\n    Płeć\n   </th>\n   <th>\n    Miejsce płeć\n   </th>\n   <th>\n    Kategoria\n   </th>\n   <th>\n    5KM\n   </th>\n   <th>\n    10KM\n   </th>\n   <th>\n    15KM\n   </th>\n   <th>\n    20KM\n   </th>\n   <th>\n    21.1KM\n   </th>\n   <th>\n    25KM\n   </th>\n   <th>\n    30KM\n   </th>\n   <th>\n    35KM\n   </th>\n   <th>\n    40KM\n   </th>\n   <th>\n    Czas netto\n   </th>\n   <th>\n    Czas brutto\n   </th>\n  </tr>\n </thead>\n <tr>\n  <td>\n   1\n  </td>\n  <td>\n   1\n  </td>\n  <t'

In [4]:
# Get head and data
raw_data_head = [[cell_value.text.strip() for cell_value in row.find_all('th')] for row in table.find_all('tr')]
raw_data = [[cell_value.text.strip() for cell_value in row.find_all('td')] for row in table.find_all('tr')]
raw_data

[[],
 ['1',
  '1',
  'KYEVA COSMAS MUTUKU',
  'NAIROBI',
  'KEN',
  '',
  'M',
  '1',
  'M30',
  '00:15:28',
  '00:31:00',
  '00:46:19',
  '01:01:56',
  '01:05:21',
  '01:17:35',
  '01:33:17',
  '01:48:42',
  '02:04:56',
  '02:11:43',
  '02:11:45.00'],
 ['2',
  '2',
  'BELACHEW ENDALE ABAYNEH',
  'ADIS ABEBA',
  'ETH',
  '',
  'M',
  '2',
  'M30',
  '00:15:28',
  '00:31:00',
  '00:46:17',
  '01:01:56',
  '01:05:20',
  '01:17:34',
  '01:33:17',
  '01:48:43',
  '02:04:55',
  '02:11:49',
  '02:11:51.00'],
 ['3',
  '10',
  'SANG SILAS KIPNGETICH',
  'KEIYO',
  'KEN',
  '',
  'M',
  '3',
  'M40',
  '00:15:28',
  '00:31:00',
  '00:46:20',
  '01:01:56',
  '01:05:20',
  '01:17:34',
  '01:33:17',
  '01:49:23',
  '02:06:42',
  '02:14:30',
  '02:14:32.00'],
 ['4',
  '15',
  'TUFA GETACHEW KENE',
  'BURA BEREH',
  'ETH',
  '',
  'M',
  '4',
  'M18',
  '00:15:28',
  '00:31:01',
  '00:46:18',
  '01:01:56',
  '01:05:20',
  '01:17:34',
  '01:33:17',
  '01:49:07',
  '02:08:02',
  '02:18:07',
  '02:18:0

In [5]:
# Make df column names
header = [row for row in raw_data_head if len(row) == 20][0]
header

['#',
 'Numer',
 'Imię i nazwisko',
 'Miasto',
 'Kraj',
 'Team',
 'Płeć',
 'Miejsce płeć',
 'Kategoria',
 '5KM',
 '10KM',
 '15KM',
 '20KM',
 '21.1KM',
 '25KM',
 '30KM',
 '35KM',
 '40KM',
 'Czas netto',
 'Czas brutto']

In [6]:
# Make df from raw_data
def make_df():
    cleaned_data = []
    for row in raw_data:
        if len(row) == 20:
            cleaned_data.append(row)
    data_frame = pd.DataFrame(data=cleaned_data, columns=header)
    return data_frame

In [7]:
df = make_df()
df[:5]

,#,Numer,Imię i nazwisko,Miasto,Kraj,Team,Płeć,Miejsce płeć,Kategoria,5KM,10KM,15KM,20KM,21.1KM,25KM,30KM,35KM,40KM,Czas netto,Czas brutto
0,1,1,KYEVA COSMAS MUTUKU,NAIROBI,KEN,,M,1,M30,00:15:28,00:31:00,00:46:19,01:01:56,01:05:21,01:17:35,01:33:17,01:48:42,02:04:56,02:11:43,02:11:45.00
1,2,2,BELACHEW ENDALE ABAYNEH,ADIS ABEBA,ETH,,M,2,M30,00:15:28,00:31:00,00:46:17,01:01:56,01:05:20,01:17:34,01:33:17,01:48:43,02:04:55,02:11:49,02:11:51.00
2,3,10,SANG SILAS KIPNGETICH,KEIYO,KEN,,M,3,M40,00:15:28,00:31:00,00:46:20,01:01:56,01:05:20,01:17:34,01:33:17,01:49:23,02:06:42,02:14:30,02:14:32.00
3,4,15,TUFA GETACHEW KENE,BURA BEREH,ETH,,M,4,M18,00:15:28,00:31:01,00:46:18,01:01:56,01:05:20,01:17:34,01:33:17,01:49:07,02:08:02,02:18:07,02:18:09.00
4,5,12,SAJI ABDELKABIR,SKHIRATE TEMARA,MAR,,M,5,M40,00:15:31,00:31:02,00:46:49,01:03:17,01:06:50,01:20:02,01:36:38,01:54:07,02:11:22,02:18:35,02:18:36.00


### Cleaning

In [8]:
# Delete useless columns, change columns names
df = df[['Miasto', 'Kraj', 'Płeć', 'Miejsce płeć', 'Kategoria', '5KM', '10KM', '15KM', '20KM', '21.1KM',
         '25KM', '30KM', '35KM', '40KM', 'Czas netto', 'Czas brutto']]
df.rename(columns={'Miasto':'City', 'Kraj':'Country', 'Płeć':'Sex', 'Miejsce płeć':'Place sex', 'Kategoria':'Cat',
                   'Czas netto':'Net time', 'Czas brutto':'Gross time'}, inplace=True)


In [9]:
df[:5]

,City,Country,Sex,Place sex,Cat,5KM,10KM,15KM,20KM,21.1KM,25KM,30KM,35KM,40KM,Net time,Gross time
0,NAIROBI,KEN,M,1,M30,00:15:28,00:31:00,00:46:19,01:01:56,01:05:21,01:17:35,01:33:17,01:48:42,02:04:56,02:11:43,02:11:45.00
1,ADIS ABEBA,ETH,M,2,M30,00:15:28,00:31:00,00:46:17,01:01:56,01:05:20,01:17:34,01:33:17,01:48:43,02:04:55,02:11:49,02:11:51.00
2,KEIYO,KEN,M,3,M40,00:15:28,00:31:00,00:46:20,01:01:56,01:05:20,01:17:34,01:33:17,01:49:23,02:06:42,02:14:30,02:14:32.00
3,BURA BEREH,ETH,M,4,M18,00:15:28,00:31:01,00:46:18,01:01:56,01:05:20,01:17:34,01:33:17,01:49:07,02:08:02,02:18:07,02:18:09.00
4,SKHIRATE TEMARA,MAR,M,5,M40,00:15:31,00:31:02,00:46:49,01:03:17,01:06:50,01:20:02,01:36:38,01:54:07,02:11:22,02:18:35,02:18:36.00


In [10]:
# Delete microseconds from Gross time column
for i in range(0, len(df)):
    df['Gross time'][i] = (df['Gross time'][i]).split('.')[0]
#df['Gross time'][:10]

In [11]:
# Add Finish column and reorder
df['Finish'] = df['Gross time']
df = df[['City', 'Country', 'Sex', 'Place sex', 'Cat', '5KM', '10KM', '15KM', '20KM', '21.1KM',
         '25KM', '30KM', '35KM', '40KM', 'Finish', 'Net time', 'Gross time']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4885 entries, 0 to 4884
Data columns (total 17 columns):
City          4885 non-null object
Country       4885 non-null object
Sex           4885 non-null object
Place sex     4885 non-null object
Cat           4885 non-null object
5KM           4885 non-null object
10KM          4885 non-null object
15KM          4885 non-null object
20KM          4885 non-null object
21.1KM        4885 non-null object
25KM          4885 non-null object
30KM          4885 non-null object
35KM          4885 non-null object
40KM          4885 non-null object
Finish        4885 non-null object
Net time      4885 non-null object
Gross time    4885 non-null object
dtypes: object(17)
memory usage: 648.9+ KB


In [12]:
# Change type of time values
date_time_str = '2018-10-14 9:00:00' # start time
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')


def times():
    for column in df.columns[5:15]:
        for i in range (0, len(df)):
            time_s = str(df[column][i])
            if time_s == '':
                df[column][i] = np.nan
            else:  
                t = time.strptime(time_s, '%H:%M:%S')
                delta = timedelta(hours=t.tm_hour, minutes=t.tm_min, seconds=t.tm_sec)
                new_date = date_time_obj + delta
                df[column][i] = new_date
    return df

df = times()
print(df[:10])

              City Country Sex Place sex  Cat                  5KM  \
0          NAIROBI     KEN   M         1  M30  2018-10-14 09:15:28   
1       ADIS ABEBA     ETH   M         2  M30  2018-10-14 09:15:28   
2            KEIYO     KEN   M         3  M40  2018-10-14 09:15:28   
3       BURA BEREH     ETH   M         4  M18  2018-10-14 09:15:28   
4  SKHIRATE TEMARA     MAR   M         5  M40  2018-10-14 09:15:31   
5          KRAŚNIK     POL   M         6  M18  2018-10-14 09:16:09   
6             SHOA     ETH   K         1  K30  2018-10-14 09:17:52   
7         TAKASAKI     JPN   M         7  M18  2018-10-14 09:17:45   
8            ARSSI     ETH   K         2  K18  2018-10-14 09:17:51   
9          SENDAFA     ETH   K         3  K18  2018-10-14 09:17:53   

                  10KM                 15KM                 20KM  \
0  2018-10-14 09:31:00  2018-10-14 09:46:19  2018-10-14 10:01:56   
1  2018-10-14 09:31:00  2018-10-14 09:46:17  2018-10-14 10:01:56   
2  2018-10-14 09:31:00  2

In [13]:
# Fill nans
df.fillna(method='pad', inplace=True) # There are not so many values, so basic mathod should fits ok.

In [14]:
# Check all column types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4885 entries, 0 to 4884
Data columns (total 17 columns):
City          4885 non-null object
Country       4885 non-null object
Sex           4885 non-null object
Place sex     4885 non-null object
Cat           4885 non-null object
5KM           4885 non-null datetime64[ns]
10KM          4885 non-null datetime64[ns]
15KM          4885 non-null datetime64[ns]
20KM          4885 non-null datetime64[ns]
21.1KM        4885 non-null datetime64[ns]
25KM          4885 non-null datetime64[ns]
30KM          4885 non-null datetime64[ns]
35KM          4885 non-null datetime64[ns]
40KM          4885 non-null datetime64[ns]
Finish        4885 non-null datetime64[ns]
Net time      4885 non-null object
Gross time    4885 non-null object
dtypes: datetime64[ns](10), object(7)
memory usage: 648.9+ KB


In [15]:
# Save df to file.
pd.DataFrame.to_csv(df, 'marathon_results.csv')